In [ ]:
import numpy as np
import pandas as pd
import gurobipy as gp

from gurobipy import GRB

from pathlib import Path
import os
import sys
from datetime import datetime, date

In [ ]:
result = Path('result/')
report = Path('report/')
instances = Path('instances/csifa')

In [ ]:
file_name = sys.argv[1]

In [ ]:
def read_instance(file_name):
	
	arq = open(file_name)
	
	N = int(arq.readline())
	
	PR  = np.zeros(N)
	PP  = np.zeros(N)

	FR = [float(arq.readline())]*N
	FP = [float(arq.readline())]*N

	HR = [float(arq.readline())]*N
	HP = [float(arq.readline())]*N

	D = [int(i) for i in arq.readline().split()]
	
	R = [int(i) for i in arq.readline().split()]

	C = float(arq.readline().rstrip('\n'))
	
	return N, PP, PR, FP, FR, HR, HP, D, R, C

In [ ]:
MAX_CPU_TIME = 3600.0
EPSILON = 1e-6

def clsr_sp_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C):

	CSP = (np.zeros((N,N))).tolist()
	CSR = (np.zeros((N,N))).tolist()
	CR = (np.zeros((N,N))).tolist()
	CL = (np.zeros(N)).tolist()

	for i in range(N):
		for j in range(i,N):
			CR[i][j]  = sum(HR[t]*SR[i][t] for t in range(i,j))
			CSP[i][j] = PP[i] * SD[i][j] + sum(HP[t]*SD[t+1][j] for t in range(i,j))
			CSR[i][j] = PR[i] * SD[i][j] + sum(HP[t]*SD[t+1][j] for t in range(i,j))

	for i in range(N):
		CL[i] = sum(HR[j]*SR[i][j] for j in range(i,N))	

	try:

		# Create a new model
		model = gp.Model("clsr_sp_mip")

		# Create variables
		zsp = model.addVars([(i,j) for i in range(N) for j in range(i,N)],vtype=GRB.CONTINUOUS, name="zsp")
		zsr = model.addVars([(i,j) for i in range(N) for j in range(i,N)],vtype=GRB.CONTINUOUS, name="zsr")
		zr = model.addVars([(i,j) for i in range(N) for j in range(i,N)],vtype=GRB.CONTINUOUS, name="zr")
		l = model.addVars(list(range(N)), lb = 0.0, ub = 1.0, vtype=GRB.CONTINUOUS, name="l")
		yp = model.addVars(list(range(N)), lb = 0.0, ub = 1.0, vtype=GRB.BINARY, name="yp")
		yr = model.addVars(list(range(N)), lb = 0.0, ub = 1.0, vtype=GRB.BINARY, name="yr")

		## Set objective
		#fobj = 0.0
		#for i in range(N):
		#	fobj += yp[i]*FP[i] + yr[i]*FR[i] + l[i]*CL[i]
        #   fobj += gp.quicksum(zsp[i,j]*CSP[i][j] + zsr[i,j]*CSR[i][j] + zr[i,j]*CR[i][j] for j in range(i,N))
		
		#model.setObjective(fobj, sense = GRB.MINIMIZE)

		## Set objective
		fobj = 0.0
		for i in range(N):
			fobj += yp[i]*FP[i] + yr[i]*FR[i] + l[i]*CL[i] 
			
			for j in range(i,N):
				fobj += zsp[i,j]*CSP[i][j] + zsr[i,j]*CSR[i][j] + zr[i,j]*CR[i][j] 

		model.setObjective(fobj, sense = GRB.MINIMIZE)

		## Add constraints

		model.addConstr(
			gp.quicksum(zsp[0,j] + zsr[0,j] for j in range(N)) == 1
			)
		
		model.addConstrs(
			gp.quicksum(zsp[i,t-1] + zsr[i,t-1] for i in range(t)) - 
			gp.quicksum(zsp[t,j] + zsr[t,j] for j in range(t, N)) == 0  for t in range(1,N) 
			)
				
		model.addConstrs(
			gp.quicksum(zsp[t,j] for j in range(t,N)) <= yp[t] for t in range(N)
			)
			
		model.addConstrs(
			gp.quicksum(zsr[t,j] for j in range(t,N)) <= yr[t] for t in range(N)
			)
			
		model.addConstr(
			gp.quicksum(zr[0,j] for j in range(N)) + l[0]==1
			)
					
		model.addConstrs(
			gp.quicksum(zr[i,t-1] for i in range(0,t)) == 
			gp.quicksum(zr[t,j]  for j in  range(t,N)) + l[t] for t in range(1,N)
			)       
				
		model.addConstrs(
			gp.quicksum(zr[i,t] for i in range(0,t+1)) <= yr[t] for t in range(N)
			)    
			
		model.addConstrs(
			gp.quicksum(SR[i][t]*zr[i,t] for i in range(t+1) ) ==
			gp.quicksum(SD[t][j]*zsr[t,j] for j in range(t,N)) for t in range(N)
			)

#		model.addConstrs(
#			gp.quicksum(SD[i][t]*zsp[i,t] for t in range(i,N)) <= yp[i]*min(C,SD[i][N-1]) for i in range(N)
#			)
		model.addConstrs(
			gp.quicksum(SD[i][t]*zsp[i,t] for t in range(i,N)) <= yp[i]*SD[i][N-1] for i in range(N)
			)
		
		model.addConstrs(
			gp.quicksum(SD[i][t]*zsr[i,t] for t in range(i,N)) <= yr[i]*SD[i][N-1] for i in range(N)
			)
		
		model.addConstrs(
			gp.quicksum(SD[t][k]*zsp[t,k] for k in range(t,N)) + 
			gp.quicksum(SD[t][k]*zsr[t,k] for k in range(t,N)) <= C for t in range(N)
			)
		
		# Parameters 
		model.setParam(GRB.Param.TimeLimit, MAX_CPU_TIME)
		model.setParam(GRB.Param.MIPGap, EPSILON)
		model.setParam(GRB.Param.Threads, 1)
		#model.setParam(GRB.Param.Cuts, -1)
		#model.setParam(GRB.Param.Presolve, -1)

		# Optimize model
		model.optimize()

		tmp = 0
		if model.status == GRB.OPTIMAL:
			tmp = 1

		print('Obj: %g' % model.ObjVal)

	except gp.GurobiError as e:
		print('Error code ' + str(e.errno) + ': ' + str(e))

	return model.ObjVal, model.ObjBound, model.MIPGap, model.Runtime, model.NodeCount, tmp


In [ ]:
def main(file_name):
	
	#file_name = f"c52_1.txt"

	N, PP, PR, FP, FR, HR, HP, D, R, C = read_instance(os.path.join(instances,file_name))

	SD = (np.zeros((N,N))).tolist()
	SR = (np.zeros((N,N))).tolist()

	for  i in range(N):
		SD[i][i] = D[i]
		SR[i][i] = R[i]
		for j in range(i+1,N):
			SD[i][j] = SD[i][j-1] + D[j]
			SR[i][j] = SR[i][j-1] + R[j]

	bestsol, bestbound, gap, runtime, numnode, status = clsr_sp_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C)
	
	arquivo = open(os.path.join(result,'clsr_sp_mip.txt'),'a')
	
	arquivo.write(file_name+';'+str(round(bestsol,2))+';'+str(round(bestbound,2))+\
					';'+str(round(gap,2))+';'+str(round(runtime,2))+';'+str(round(numnode,3))+\
					';'+str(round(status,2))+
					'\n')

	arquivo.close()

In [ ]:
if __name__== "__main__" :

	file_name = f"c52_1.txt"

	main(file_name)